# Introduction to scalable computing with Dask

## What is Dask?

* Parallel and distributed computing (diagramatic rep of both)
* Various API, distributed scheduler
* We'll look at Dask DataFrame

## Dask DataFrame API

* Parallel pandas
* Discuss parititions
* Discuss paradigm shift with mean and median example

## Dask cluster

* Scheduler, workers, dashboard

## Parallelize previous pandas workflows

- LocalCluster?
- Parallel read (CSV vs Parquet)
- Previous exercises

In [1]:
import json
import gcsfs

In [2]:
token = json.load(open("prep/credentials.json"))
fs = gcsfs.GCSFileSystem(token=token)
storage_options={"token": token}

In [3]:
from dask.distributed import LocalCluster, Client

In [4]:
cluster = LocalCluster(n_workers=2)

In [5]:
client = Client(cluster)

In [6]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 4,Total memory: 14.65 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46167,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 14.65 GiB
Comm: tcp://127.0.0.1:37227,Total threads: 2
Dashboard: http://127.0.0.1:34711/status,Memory: 7.32 GiB
Nanny: tcp://127.0.0.1:35793,


https://nebari.quansight.dev/user/login_email/proxy/8787/

### Read the entire dataset

In [7]:
import dask.dataframe as dd

In [9]:
%% time

ddf = dd.read_csv("gcs://quansight-datasets/airline-ontime-performance/csv/*",
                 storage_options = storage_options)

In [ ]:
ddf["flight]

In [10]:
ddf

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,DIV_AIRPORT_LANDINGS,DIV_REACHED_DEST,DIV_ACTUAL_ELAPSED_TIME,DIV_ARR_DELAY,DIV_DISTANCE,DIV1_AIRPORT,DIV1_AIRPORT_ID,DIV1_AIRPORT_SEQ_ID,DIV1_WHEELS_ON,DIV1_TOTAL_GTIME,DIV1_LONGEST_GTIME,DIV1_WHEELS_OFF,DIV1_TAIL_NUM,DIV2_AIRPORT,DIV2_AIRPORT_ID,DIV2_AIRPORT_SEQ_ID,DIV2_WHEELS_ON,DIV2_TOTAL_GTIME,DIV2_LONGEST_GTIME,DIV2_WHEELS_OFF,DIV2_TAIL_NUM,DIV3_AIRPORT,DIV3_AIRPORT_ID,DIV3_AIRPORT_SEQ_ID,DIV3_WHEELS_ON,DIV3_TOTAL_GTIME,DIV3_LONGEST_GTIME,DIV3_WHEELS_OFF,DIV3_TAIL_NUM,DIV4_AIRPORT,DIV4_AIRPORT_ID,DIV4_AIRPORT_SEQ_ID,DIV4_WHEELS_ON,DIV4_TOTAL_GTIME,DIV4_LONGEST_GTIME,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
npartitions=624,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int64,int64,int64,object,object,int64,object,float64,int64,int64,int64,int64,object,object,object,int64,object,int64,int64,int64,int64,object,object,object,int64,object,int64,int64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

Lazy evaluation

In [11]:
ddf.head() # Will raise an error

2023-04-08 14:59:33,654 - distributed.worker - WARNING - Compute Failed
Key:       ('read-csv-ec8707a5581123dabf6af00e0c05775e', 0)
Function:  execute_task
args:      ((subgraph_callable-5e719f03-45b1-4027-9609-3104273a655c, [(<function read_block_from_file at 0x7f39bb144dc0>, <OpenFile 'quansight-datasets/airline-ontime-performance/csv/bts_airline_ontime_performance_april_2003.csv'>, 0, 65257922, b'\n'), None, True, False]))
kwargs:    {}
Exception: 'ValueError(\'Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.\\n\\n+----------+--------+----------+\\n| Column   | Found  | Expected |\\n+----------+--------+----------+\\n| TAIL_NUM | object | float64  |\\n+----------+--------+----------+\\n\\nThe following columns also raised exceptions on conversion:\\n\\n- TAIL_NUM\\n  ValueError("could not convert string to float: \\\'N050AA\\\'")\\n\\nUsually this is due to dask\\\'s dtype inference failing, and\\n*may* be fixed by specifying dtypes manually by adding:\\n\\ndtype={\\\'TAIL_

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+----------+--------+----------+
| Column   | Found  | Expected |
+----------+--------+----------+
| TAIL_NUM | object | float64  |
+----------+--------+----------+

The following columns also raised exceptions on conversion:

- TAIL_NUM
  ValueError("could not convert string to float: 'N050AA'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'TAIL_NUM': 'object'}

to the call to `read_csv`/`read_table`.

### Longest flight (distance) across the dataset

In [14]:
ddf["DISTANCE"].max()

dd.Scalar<series-..., dtype=float64>

In [ ]:
# takes ~8 mins on a Medium instance

ddf["DISTANCE"].max().compute()

### Specify `dtypes`

Dask infers the dtypes using the first row, because it's "lazy" and does not read the entire dataset.

Best practice: explicit dtypes

In [12]:
with open('prep/dtypes.json', 'r') as f:
    dtypes = json.load(f)

In [13]:
ddf = dd.read_csv("gcs://quansight-datasets/airline-ontime-performance/csv/*",
                 storage_options = storage_options,
                 dtypes=dtypes)

TypeError: read_csv() got an unexpected keyword argument 'dtypes'

In [ ]:
ddf.head()

In [16]:
client.close()
cluster.close()

## Dask Gateway

* What it is?
* How it works in Nebari (diagram?)

- Intro to dashboard
- First time sign in

In [12]:
import dask_gateway

gateway = dask_gateway.Gateway()

In [13]:
options = gateway.cluster_options(use_local_defaults=False)
options

In [14]:
cluster = gateway.new_cluster(options)
cluster

In [15]:
client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://nebari.quansight.dev/gateway/clusters/dev.813339dafcb04264b19f5dcd23540787/status,


2023-04-08 08:36:35,618 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


First time sign-in

## Adaptive scaling

### Always make sure to shutdown your cluster

In [ ]:
client.close(shutdown=True)

## Other Dask APIs

* Dask Array
* Dask Bag
* Dask Delayed and Futures

Check out the official Dask tutorial to learn more!